In [1]:
# Lasso回帰はL1正則化回帰とも呼ばれる
# Ridge回帰はL2正則化回帰とも呼ばれる

# Lasso回帰では、尖っている部分（微分不可能な点で、特定の回帰係数の大きさが0となっている）が解になりやすくなるため、いくつかの係数が0となる解が推定されやすい。
# Ridge回帰では、制約条件が円状であるため、回帰係数が全体的に小さくなる領域が解になりやすい

# Lasso回帰は不要なパラメータを削りたい（次元・特徴量削減）という時によく使われる
# Ridge回帰は過学習を抑えて汎化された滑らかなモデルを得やすい事がしらてている

import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import linear_model
%precision 3
%matplotlib inline